### Created by: Anthony D. Cho
### Last update: 13.10.2021

**Subject**: Linear regression (OLR, Ridge, Lasso) - Applied to Seoul Bike Sharing Demand


## Libraries dependencies

In [1]:
import warnings
warnings.filterwarnings('ignore')

from ipywidgets import interact#, interact_manual

import matplotlib.pyplot as plt
from pandas import read_csv, get_dummies

## Pre-processing functions
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from numpy import where

## Models
from sklearn.linear_model import LinearRegression, Ridge, Lasso

## Metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

%matplotlib inline

# Problem: Seoul Bike Sharing Demand

**Target**: Predict the Rented Bike Count

## Data loading

Source: [Seoul Bike Sharing Demand Data Set](https://archive.ics.uci.edu/ml/datasets/Seoul+Bike+Sharing+Demand) (UCI Repository)

In [2]:
## Load data
data = read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/00560/SeoulBikeData.csv',
               encoding='latin_1', parse_dates=['Date'])

In [3]:
## Data type
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Date                       8760 non-null   datetime64[ns]
 1   Rented Bike Count          8760 non-null   int64         
 2   Hour                       8760 non-null   int64         
 3   Temperature(°C)            8760 non-null   float64       
 4   Humidity(%)                8760 non-null   int64         
 5   Wind speed (m/s)           8760 non-null   float64       
 6   Visibility (10m)           8760 non-null   int64         
 7   Dew point temperature(°C)  8760 non-null   float64       
 8   Solar Radiation (MJ/m2)    8760 non-null   float64       
 9   Rainfall(mm)               8760 non-null   float64       
 10  Snowfall (cm)              8760 non-null   float64       
 11  Seasons                    8760 non-null   object        
 12  Holida

In [4]:
## data description
data.describe(include='all')

,Date,Rented Bike Count,Hour,Temperature(°C),Humidity(%),Wind speed (m/s),Visibility (10m),Dew point temperature(°C),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Seasons,Holiday,Functioning Day
count,8760,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760,8760,8760
unique,365,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,2,2
top,2018-05-21 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Summer,No Holiday,Yes
freq,24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2208,8328,8465
first,2017-01-12 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,2018-12-11 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,704.602055,11.500000,12.882922,58.226256,1.724909,1436.825799,4.073813,0.569111,0.148687,0.075068,NaN,NaN,NaN
std,NaN,644.997468,6.922582,11.944825,20.362413,1.036300,608.298712,13.060369,0.868746,1.128193,0.436746,NaN,NaN,NaN
min,NaN,0.000000,0.000000,-17.800000,0.000000,0.000000,27.000000,-30.600000,0.000000,0.000000,0.000000,NaN,NaN,NaN
25%,NaN,191.000000,5.750000,3.500000,42.000000,0.900000,940.000000,-4.700000,0.000000,0.000000,0.000000,NaN,NaN,NaN


In [5]:
data.head(4)

,Date,Rented Bike Count,Hour,Temperature(°C),Humidity(%),Wind speed (m/s),Visibility (10m),Dew point temperature(°C),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Seasons,Holiday,Functioning Day
0,2017-01-12,254,0,-5.2,37,2.2,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
1,2017-01-12,204,1,-5.5,38,0.8,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
2,2017-01-12,173,2,-6.0,39,1.0,2000,-17.7,0.0,0.0,0.0,Winter,No Holiday,Yes
3,2017-01-12,107,3,-6.2,40,0.9,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes


In [6]:
data['Seasons'].value_counts()

Summer    2208
Spring    2208
Autumn    2184
Winter    2160
Name: Seasons, dtype: int64

In [7]:
where(data['Holiday'] == 'Holiday', 1, 0)

array([0, 0, 0, ..., 0, 0, 0])

## Data pre-processing

In [8]:
## Drop Date column
data.drop(columns=['Date'], inplace=True)

## Encoding: Holiday column in numeric values (0: No Holiday, 1: Holiday)
data['Holiday'] = where(data['Holiday'] == 'Holiday', 1, 0)

## Encoding: Functioning Day column in numeric values (0: No, 1: Yes)
data['Functioning Day'] = where(data['Functioning Day'] == 'Yes', 1, 0)

## Encoding: Seasons column using One-Hot encoding
data = get_dummies(data, drop_first=True)

In [9]:
## Data partition (hold-out validation)
trainValSet, testSet = train_test_split(data, train_size=0.85, random_state=0)
trainSet, valSet = train_test_split(trainValSet, train_size=0.85, random_state=0)

## Data standardization (this function return a numpy.ndarray)
scaler = StandardScaler().fit(trainSet)
trainSet_scaled = scaler.transform(trainSet)
valSet_scaled = scaler.transform(valSet)
testSet_scaled = scaler.transform(testSet)

## Predictors and target
X_train, y_train = trainSet_scaled[:, 1:], trainSet_scaled[:, 0]
X_val,   y_val =   valSet_scaled[:, 1:], valSet_scaled[:, 0]
X_test,  y_test =  testSet_scaled[:, 1:], testSet_scaled[:, 0]

In [10]:
print('(train shape) X: {}, y: {}'.format(X_train.shape, y_train.shape))
print('(Validation shape) X: {}, y: {}'.format(X_val.shape, y_val.shape))
print('(test shape) X: {}, y: {}'.format(X_test.shape, y_test.shape))

(train shape) X: (6329, 14), y: (6329,)
(Validation shape) X: (1117, 14), y: (1117,)
(test shape) X: (1314, 14), y: (1314,)


## Model: Ordinary Linear Regression (OLR)

In [11]:
## Model building
model = LinearRegression(n_jobs=-1).fit(X_train, y_train)
print('Train set score: {:.4f}'.format(model.score(X_train, y_train)))
print('Test set score: {:.4f}'.format(model.score(X_test, y_test)))

## Prediction using validation set
prediction = model.predict(X_val)
print('MSE (Val): {:.4f}'.format(mean_squared_error(y_true=y_val, y_pred=prediction)) )
print('MAE (Val): {:.4f}'.format(mean_absolute_error(y_true=y_val, y_pred=prediction)) )
print('R2  (Val): {:.4f}'.format(r2_score(y_true=y_val, y_pred=prediction)), end='\n'*2 )

## Prediction using test set
prediction = model.predict(X_test)
print('MSE (Test): {:.4f}'.format(mean_squared_error(y_true=y_test, y_pred=prediction)) )
print('MAE (Test): {:.4f}'.format(mean_absolute_error(y_true=y_test, y_pred=prediction)) )
print('R2  (Test): {:.4f}'.format(r2_score(y_true=y_test, y_pred=prediction)) )

Train set score: 0.5543
Test set score: 0.5375
MSE (Val): 0.4475
MAE (Val): 0.4924
R2  (Val): 0.5413

MSE (Test): 0.4792
MAE (Test): 0.5118
R2  (Test): 0.5375


## Model: Ridge / LASSO regression

In [12]:
## features names
featuresName = data.columns[1:]

## Scores storage
scores = {'Ridge':{'Alpha':[], 'MSE': [], 'MAE':[], 'R2': []}, 
          'Lasso':{'Alpha':[], 'MSE': [], 'MAE':[], 'R2': []}
         }

@interact
def plotRegularizationModel(alpha=(0.000001, 0.7, 0.001)):
    plt.figure(figsize=(13, 10))
    
    ## Models instance
    modelName = ['Ridge', 'Lasso']
    models = [Ridge(alpha=alpha), Lasso(alpha=alpha)]

    i = 1
    for name, model in zip(modelName, models):
        
        ## Model fitting
        model.fit(X_train, y_train)
        prediction = model.predict(X_val)
        
        ## Computing scores
        scores[name]['Alpha'].append( alpha )
        scores[name]['R2'].append( model.score(X_val, y_val) )
        scores[name]['MSE'].append( mean_squared_error(y_true=y_val, y_pred=prediction) )
        scores[name]['MAE'].append( mean_absolute_error(y_true=y_val, y_pred=prediction) )
        
        ## Plot model's coefficients
        plt.subplot(2, 2, 2*i-1)
        plt.bar(range(len(model.coef_)), model.coef_, width=0.08, color='r')
        plt.axhline(color='k')
        plt.xticks(ticks=range(len(model.coef_)), labels=featuresName, rotation=90)
        plt.xlabel('Features')
        plt.ylabel('Coef. Values')
        plt.title('Coefficients value (Model: {})'.format(name))
        plt.ylim(-0.6, 0.6)
        plt.grid()
        
        ## Plot validation scores
        plt.subplot(2, 2, 2*i)
        plt.plot(scores[name]['Alpha'], scores[name]['MSE'], label='MSE', color='red', marker='o', ls='')
        plt.plot(scores[name]['Alpha'], scores[name]['MAE'], label='MAE', color='blue', marker='o', ls='')
        plt.plot(scores[name]['Alpha'], scores[name]['R2'], label='R2', color='green', marker='o', ls='')
        plt.xlabel('Alpha')
        plt.ylabel('scores')
        plt.xlim(-0.01, 0.71)
        #plt.ylim(min(scores[nombre]['Y']) - 0.05, max(scores[name]['Y']) + 0.05)
        plt.title('Validation set scores ({})'.format(name))
        plt.legend()
        
        i += 1
    
    plt.tight_layout()
    plt.show()

interactive(children=(FloatSlider(value=0.349001, description='alpha', max=0.7, min=1e-06, step=0.001), Output…

## Best model: Rigde and LASSO

In [13]:
## Data partition (hold-out validation)
trainSet, testSet = train_test_split(data, train_size=0.85, random_state=0)

## Data standardization (this function return a numpy.ndarray)
scaler = StandardScaler().fit(trainSet)
trainSet_scaled = scaler.transform(trainSet)
testSet_scaled = scaler.transform(testSet)

## Predictors and target
X_train, y_train = trainSet_scaled[:, 1:], trainSet_scaled[:, 0]
X_test,  y_test =  testSet_scaled[:, 1:], testSet_scaled[:, 0]

In [14]:
alpha = 0.000001

In [15]:
## Build, fit and predict
model = Ridge(alpha=alpha)
model.fit(X_train, y_train)
prediction = model.predict(X_test)

print('Ridge:')
print('R2  (test): {:.4f}'.format( model.score(X_test, y_test) ))
print('MSE (test): {:.4f}'.format( mean_squared_error(y_true=y_test, y_pred=prediction) ))
print('MAE (test): {:.4f}'.format( mean_absolute_error(y_true=y_test, y_pred=prediction) ))

Ridge:
R2  (test): 0.5369
MSE (test): 0.4813
MAE (test): 0.5122


In [16]:
## Build, fit and predict
model = Lasso(alpha=alpha)
model.fit(X_train, y_train)
prediction = model.predict(X_test)

print('LASSO:')
print('R2  (test): {:.4f}'.format( model.score(X_test, y_test) ))
print('MSE (test): {:.4f}'.format( mean_squared_error(y_true=y_test, y_pred=prediction) ))
print('MAE (test): {:.4f}'.format( mean_absolute_error(y_true=y_test, y_pred=prediction) ))

LASSO:
R2  (test): 0.5369
MSE (test): 0.4813
MAE (test): 0.5122


In [17]:
## Build, fit and predict
model = LinearRegression(n_jobs=-1).fit(X_train, y_train)
prediction = model.predict(X_test)

print('OLR:')
print('R2  (test): {:.4f}'.format( model.score(X_test, y_test) ))
print('MSE (test): {:.4f}'.format( mean_squared_error(y_true=y_test, y_pred=prediction) ))
print('MAE (test): {:.4f}'.format( mean_absolute_error(y_true=y_test, y_pred=prediction) ))

OLR:
R2  (test): 0.5369
MSE (test): 0.4813
MAE (test): 0.5122
